## Call Boa<sub>g</sub> script form Python

### How has the average number of exons per gene in a species clade changed for genomes deposited before and after 2016?

In [12]:
import subprocess
import os


def run_boa(boa_script, output_directory):
    with open("test.boa", "w") as boa_file:
        boa_file.write(boa_script)
    bashCommand = "bash runBoaG.sh test.boa dataset/ " + output_directory
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

    if (len(output) < 400):
        print("error in compile step, please see the console")
    else:
        print ("Output location " + output_directory + "/part-r-00000 \n")

        bashCommand="head -10 " + output_directory +"/part-r-00000"
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        
        # print the output
        out=str(output).split("\\n")
        out=out[2:len(out)-1]
        for line in out:
            print(line)

#### Submit Boa<sub>g</sub> script as a boa_script varible

In [13]:
boa_script = """
g: Genome = input;
geneCounts: output sum[string][string] of int;
exonCounts: output sum[string][string] of int;
adata := getAssembler(g.refseq);
asYear :=yearof(adata.assembly_date);
if(asYear < 2016){
        foreach(i:int; def(g.sequence[i])){
          fdata := getFeature(g.refseq, g.sequence[i].accession);
          foreach(j:int; def(fdata.feature[j])){
                  if (match("gene",fdata.feature[j].ftype))
                        geneCounts [g.refseq][g.taxid]<< 1;  
                  if (match("exon",fdata.feature[j].ftype))
                        exonCounts [g.refseq][g.taxid]<< 1;
          }
        }  
}       

"""

output_location= "boa_output"

run_boa(boa_script, output_location)


Output location boa_output/part-r-00000 

exonCounts[GCF_000002515.2][28985] = 5547
exonCounts[GCF_000002525.2][4952] = 8537
exonCounts[GCF_000002545.3][284593] = 5629
exonCounts[GCF_000002655.1][330879] = 28488
exonCounts[GCF_000002715.2][344612] = 28194
exonCounts[GCF_000002855.3][5061] = 36537
exonCounts[GCF_000002945.1][4896] = 12264
exonCounts[GCF_000003125.1][441959] = 44642


### Show 20 lines of Boa<sub>g</sub> output

In [14]:
# read more lines or the whole output file
bashCommand="head -20 " + output_location +"/part-r-00000"

list = os.popen(bashCommand).read()
print(list)

exonCounts[GCF_000001985.1][441960] = 34417
exonCounts[GCF_000002495.2][242507] = 36522
exonCounts[GCF_000002515.2][28985] = 5547
exonCounts[GCF_000002525.2][4952] = 8537
exonCounts[GCF_000002545.3][284593] = 5629
exonCounts[GCF_000002655.1][330879] = 28488
exonCounts[GCF_000002715.2][344612] = 28194
exonCounts[GCF_000002855.3][5061] = 36537
exonCounts[GCF_000002945.1][4896] = 12264
exonCounts[GCF_000003125.1][441959] = 44642
exonCounts[GCF_000003515.1][336963] = 24143
exonCounts[GCF_000003835.1][306902] = 6168
exonCounts[GCF_000003855.2][559298] = 32155
exonCounts[GCF_000004155.1][653667] = 11468
exonCounts[GCF_000006275.2][332952] = 38122
exonCounts[GCF_000006335.3][294747] = 6475
exonCounts[GCF_000006445.2][4959] = 7026
exonCounts[GCF_000026365.1][4956] = 5543
exonCounts[GCF_000026945.1][573826] = 6351
exonCounts[GCF_000027005.1][644223] = 5618



## Post-processing with python

*  So far we used Boa<sub>g</sub> to run the most time consuming part on top of Hadoop or in a single computer, the rest of the post processing we will use Python to generate the final results, i.e, mean and standard deviation.

* The Python code is in the current directory in  a file named exon_gene.py .

In [18]:
# read more lines or the whole output file
bashCommand="python exon_gene.py " + output_location +"/part-r-00000"

list = os.popen(bashCommand).read()
print(list)

{}
#############################################
tax id: {4751: 'Fungi'}
(50, 5)
               taxid          gene           exon  exonpergene
count      50.000000     50.000000      50.000000    50.000000
mean   363313.460000   9360.460000   28536.200000     2.679020
std    200039.343669   4193.473195   22919.481263     1.570024
min      4896.000000   2010.000000    2067.000000     0.989000
25%    283880.500000   6084.750000    6382.000000     1.061000
50%    334957.500000   8858.500000   31687.000000     2.764500
75%    475020.750000  12149.500000   39360.250000     3.389000
max    876142.000000  21354.000000  100216.000000     6.620000
#############################################
tax id: {4890: 'Ascomycota'}
(42, 5)
               taxid          gene          exon  exonpergene
count      42.000000     42.000000     42.000000    42.000000
mean   352095.000000   9387.380952  24528.880952     2.300310
std    196130.289619   3817.406149  17886.039010     1.039969
min      4896.000000 